In [1]:
import os

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\Kidney_disease-DL-\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\HP\\Desktop\\Kidney_disease-DL-'

In [1]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
import sys
sys.path.append(r'C:\Users\HP\Desktop\Kidney_disease-DL-\src')


In [28]:
from cnnClassifier.constants import *


In [29]:
from cnnClassifier.utils.common import read_yaml, create_directories

In [57]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)  # This returns a dictionary
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']  # Accessing 'data_ingestion' as a dictionary

        create_directories([config['root_dir']])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_URL=config['source_URL'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )

        return data_ingestion_config


In [58]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [59]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [60]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-20 17:05:26,458: INFO: 986331348: Downloading data from https://drive.google.com/file/d/13YhEwSVr5Dpf-L0b1flBmzzGel-bdPtj/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=13YhEwSVr5Dpf-L0b1flBmzzGel-bdPtj
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=13YhEwSVr5Dpf-L0b1flBmzzGel-bdPtj&confirm=t&uuid=508c6c37-b60d-4e69-8afc-f2aaf6a2dede
To: c:\Users\HP\Desktop\Kidney_disease-DL-\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [04:36<00:00, 3.40MB/s]


[2025-01-20 17:10:08,841: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/13YhEwSVr5Dpf-L0b1flBmzzGel-bdPtj/view?usp=sharing into file artifacts/data_ingestion/data.zip]
